<a href="https://colab.research.google.com/github/Jagdish05/Fake-News-Detection/blob/main/2_Layer_NN_(7k_data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
downloaded = drive.CreateFile({'id':"1DV4wHjJ-vBOL1pt-l58OpG9ipk2-1gGf"})   # replace the id with id of file you want to access
downloaded.GetContentFile('news.csv')

In [ ]:
import pandas as pd
data = pd.read_csv('news.csv')
del data['Unnamed: 0']
data['article'] = data['title'] + ': ' + data['text']
del data['title']
del data['text']

data['label'] = data['label'].replace({'REAL': 1})
data['label'] = data['label'].replace({'FAKE': 0})

In [ ]:
articles = data['article'].values
y = data['label'].values

from sklearn.model_selection import train_test_split
articles_train, articles_test, y_train, y_test = train_test_split(articles, y, test_size=0.25, random_state=1000)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(articles_train)

X_train = vectorizer.transform(articles_train)
X_test  = vectorizer.transform(articles_test)

In [ ]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 10)                604900    
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 11        
Total params: 604,911
Trainable params: 604,911
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train, y_train, epochs=100, verbose=False, batch_size=10)

In [ ]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 1.0000
Testing Accuracy:  0.9053
